##◢ Setup

In [ ]:
#@title Setup DeOldify {display-mode: "form"}
#@markdown Click the play button on the left to run this block.
#@markdown
#@markdown This downloads the DeOldify repository and installs dependencies.
#@markdown
#@markdown Downloading the models can take some time.
#@markdown
#@markdown This block must be re-run if the runtime expires, usually because of inactivity.

#@markdown Tips:
#@markdown - Edit -> Clear all ouputs
#@markdown - Edit -> Show/hide code
#@markdown - Runtime -> Reset all runtimes
#@markdown - Runtime -> Manage sessions

def setup():
    print("Setting up filesystem")
    %cd -q /content
    %mkdir /content/upload_temp /content/original_images /content/result_images /content/download_temp &> /dev/null
    !rm sample_data -rf
    
    print("Downloading DeOldify")
    !git clone -q https://github.com/jantic/DeOldify.git DeOldify 
    %cd -q /content/DeOldify
    
    print("Installing dependencies")
    !pip install -q -r colab_requirements.txt &>/dev/null
    !apt install -qq zip &> /dev/null

    print("Downloading models")
    %mkdir models &>/dev/null
    # Downloads "Artistic" model
    !wget wget --show-progress -q https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth
    # Downloads "Stable" model
    !wget wget --show-progress -q https://data.deepai.org/deoldify/ColorizeStable_gen.pth -O ./models/ColorizeStable_gen.pth

    !touch /content/setup_finished
    
#@markdown ### Options

#@markdown force_install allows rerunning the setup.
#@markdown
#@markdown Default: False
force_install = False #@param {type:"boolean"}

import os

if not os.path.isfile("/content/setup_finished") or force_install:
    setup()
    print("Setup finished.")
else:
    print("Setup already finished.")

In [ ]:
#@title ## Upload files {display-mode: "form"}
#@markdown After running this block, a file upload box will show. One or multiple .jpg's or .zip's can be submitted.
#@markdown
#@markdown **Warning: this will remove all current uploads and results!**
#@markdown
#@markdown *Tip: use the "Files" tab on the left to see all relevant files in the "uploaded" and "results" folders.*

from google.colab import files
import shutil

def upload_files_dialog():
    %cd -q /content
    !rm -rf /content/upload_temp/* /content/original_images/* /content/DeOldify/result_images/* /content/DeOldify/download_temp/* &>/dev/null
    %cd -q /content/upload_temp
    try:
        uploaded = files.upload()
        for filename in uploaded.keys():
            if filename[-4:] == ".zip":
                !unzip -df /content/original_images/ {filename}
            elif filename[-4:] == ".jpg":
                shutil.copy(filename, "/content/original_images/")
        !rm -rf /content/upload_temp/*
        %cd -q /content/DeOldify
        print("Upload finished")
    except:
        print("Browser session expired. Please rerun this block.")

upload_files_dialog()

In [ ]:
#@title ## Colorize script {display-mode: "form"}
#@markdown This block applies colorization on the uploaded files.
#@markdown The results can be downloaded using the block below.

# Form
#@markdown ### Options

#@markdown Enable this to use the "Artistic" model (recommended). This model produces more vibrant colours.
#@markdown
#@markdown Disable to use the "Stable" model. This will often tend to do better on portraits, and natural landscapes.
#@markdown
#@markdown `Default: True`
artistic = True #@param {type:"boolean"}

#@markdown The render factor determines resolution at which the color portion of the image is rendered. Lower resolution will render faster, and colors also tend to look more vibrant.
#@markdown Older and lower quality images in particular will generally benefit by lowering the render factor.
#@markdown
#@markdown The default value of 35 has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-).
#@markdown Higher render factors are often better for higher quality images, but the colors may get slightly washed out.
#@markdown
#@markdown `Default: 35`
render_factor = 35  #@param {type: "slider", min: 7, max: 45}

#@markdown Shows full sized image in the notebook (probably won't fit the screen).
#@markdown
#@markdown `Default: False`
show_fullsize = False #@param {type:"boolean"}

#@markdown Show a comparison of the original and colored image.
#@markdown
#@markdown `Default: False`
compare = False #@param {type:"boolean"}

#@markdown Generate images for every possible render factor. Recommended to use this with one image at a time.
#@markdown
#@markdown `Default: False`
iterate = False #@param {type:"boolean"}

%cd -q /content/DeOldify

import os
if not os.path.isfile("/content/setup_finished"):
    print("Setup did not complete! Run the Setup block above and try again.")

from deoldify import device
from deoldify.device_id import DeviceId
device.set(device=DeviceId.GPU0)
import torch
import fastai
from deoldify.visualize import *

from pathlib import Path
import psutil
import gc
import warnings

warnings.simplefilter("ignore")

# Functions

def clean_mem():
    torch.cuda.empty_cache()
    n = 2**14
    a_2GB = np.ones((n, n))  # RAM: +2GB
    del a_2GB  # RAM: -2GB
    gc.collect()

def colorize(paths, colorizer):
    for path in paths:
        original_path = str(path)
        results_path = Path("/content/result_images")
        if iterate:
            for i in range(10,46,2):
                image_path = colorizer.plot_transformed_image(path=original_path, results_dir=results_path, render_factor=i, display_render_factor=True, figsize=(8,8), compare=compare, watermarked=False)
                os.rename(str(image_path), str(image_path)[:-3] + '_' + str(render_factor) + '.jpg')
        else:
            image_path = colorizer.plot_transformed_image(path=original_path, results_dir=results_path, render_factor=render_factor, compare=compare, watermarked=False)
            print(image_path)
            if show_fullsize:
                show_image_in_notebook(image_path)
        clean_mem()
        print("Memory in use:", psutil.virtual_memory().percent, "%")

# Actual script

paths = list(Path("/content/original_images").glob("**/*.jpg"))
colorizer = get_image_colorizer(artistic=artistic)

print("DeOldify will use these files:")
print(paths)
print()

!rm -rf /content/DeOldify/result_images/*

colorize(paths, colorizer)

In [ ]:
#@title Download generated results {display-mode: "form"}

from google.colab import files
from pathlib import Path
from datetime import datetime

def get_timestamp():
    timestamp = !date +%Y-%m-%d_%H-%M-%S
    return timestamp[0]

def get_result_image_paths():
    return Path("/content/result_images").glob("**/*.jpg")

def get_result_image_count():
    files = list(get_result_image_paths())
    return len(list(files))

def zip_result():
    zipfile = "/content/download_temp/DeOldify_" + get_timestamp() + ".zip"
    !zip -r {zipfile} /content/result_images/*
    return zipfile


def download_result(zip_results):
    if get_result_image_count() > 0:
        if not zip_results == "always" and (zip_results == "never" or get_result_image_count() == 1):
            for image in get_result_image_paths():
                try:
                    files.download(str(image))
                except:
                    print("\033[1mBrowser session expired. Please rerun this block.\033[0m")
        elif zip_results == "always" or get_result_image_count() > 1:
            zipfile = zip_result()
            print("Downloading result")
            try:
                files.download(zipfile)
            except:
                !rm -rf {zipfile}
                print("\033[1mBrowser session expired. Please rerun this block.\033[0m")
        
#@markdown ### Options
#@markdown auto = more than 1 result will get zipped, single files will just get downloaded
#@markdown
#@markdown never = download each result individually
#@markdown
#@markdown always = always zip results
#@markdown
#@markdown Default: auto
zip_results = "auto" #@param ["auto", "never", "always"]

download_result(zip_results)

In [ ]:
#@title Delete uploaded and generated files {display-mode: "form"}
#@markdown Useful if you ran blocks in the wrong order.
!rm -rf /content/uploaded_temp/*
!rm -rf /content/original_images/*
!rm -rf /content/result_images/*
!rm -rf /content/download_temp/*